**CNN.py** Trains and tests a Convolutional Neural Network using the ImageNet ResNet-50

In [ ]:
from tensorflow.keras.layers import Dense, BatchNormalization, GlobalAveragePooling2D, Dropout, Activation
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import pandas as pd
import numpy as np
import os

os.chdir(r"C:\Carabid_Data\Invert")

Importing modules <br>
Set dataset directory (adjust this to your own directory)

In [ ]:
df = pd.read_csv("shuffletrain.csv")

Y = df['AllTaxa']
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

Read in feature vector dataset <br>
Labels ('AllTaxa' for LITL dataset or 'Order' for order level dataset) are converted to one hot encoded labels as dummy_y <br>
The numeric data is not used for this model

In [ ]:
num_class = dummy_y.shape[1]

base_model = ResNet50(include_top = False, weights = 'imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)
predict = Dense(num_class, activation = "softmax")(x)
resnet = Model(inputs = base_model.input, outputs = predict)

for layer in base_model.layers:
    layer.trainable = False
    
resnet.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

The Image Net ResNet50 is used as the base model, with a global average pooling layer between it and the dense layers. Two dense layers are added with batch normalization and 0.3 dropout. The size of the ouput layer is determined by the number of classes in `dummy_y`. The ResNet layers are set to not be trainable.

In [ ]:
resnet.fit(
    x=images, y=dummy_y,
    epochs=10, batch_size=128,
    verbose = 1)

The model is fit using images generated in **LoadImages.py**

In [ ]:
preds = resnet.predict(testimages)

The model is tested using images generated in **LoadImages.py**